<a href="https://colab.research.google.com/github/vicky1412/Predicting_stocks_usiang_deep_learning-models/blob/main/model_1m(price).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [ ]:
import datetime
import time
import math
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense,LSTM
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

## Data

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Option Trading/EurUsd-1m.csv')

In [ ]:
data.head()

In [ ]:
df = data.filter(['Close'])

#convert dataframe to a numpy array
dataset = df.values
#Get the no of rows to train the model on
training_data_len = math.ceil(len(dataset)*0.8)

## Scale between(0,1)

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)

## create the train data set

In [ ]:
train_data = scaled_data[0:training_data_len]

# split the data into x_train and y_train data set
x_train = []
y_train = []

# x_train = 60 values, y_train = 61st and 62nd value
for i in range(60,len(train_data)):
  x_train.append(train_data[i-60:i])
  y_train.append(train_data[i])

#convert x_train and y_train into numpy arrrays
x_train,y_train = np.array(x_train),np.array(y_train)

# #Reshape x_train to 3 dim because LSTM requires 3 dim
# x_train = np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))

In [ ]:
x_train.shape, y_train.shape, x_train.dtype, y_train.dtype

((24988, 60, 1), (24988, 1), dtype('float64'), dtype('float64'))

## Create Test dataset

In [ ]:
test_data = scaled_data[training_data_len-60:]

#split the data into x_test and y_test data set
x_test = []
y_test = dataset[training_data_len:]

#x_test = 60 values, y_test = 61st and 62nd value
for i in range(60,len(test_data)):
  x_test.append(test_data[i-60:i])

#convert x_test and y_test into numpy arrrays
x_test = np.array(x_test)

# #Reshape x_test to 3 dim because LSTM requires 3 dim
# x_test = np.reshape(x_test,(x_test.shape[0],x_test.shape[1],1))

In [ ]:
x_test.shape, type(y_test), x_test.dtype, len(y_test)

((6261, 60, 1), numpy.ndarray, dtype('float64'), 6261)

## Build the LSTM Model

In [ ]:
model = Sequential()
model.add(LSTM(50, return_sequences=True))
model.add(LSTM(50, return_sequences=False, input_shape = (x_train.shape[1],1)))
model.add(Dense(25))
model.add(Dense(1))

In [ ]:
#compile the model
model.compile(optimizer='adam',loss='mean_squared_error')

## train the model

In [ ]:
np.random.seed(42)
model.fit(x_train,y_train,batch_size=1,epochs=1)

24988/24988 [==============================] - 504s 19ms/step - loss: 4.1156e-04


In [ ]:
from keras.models import load_model
model = load_model('/content/drive/MyDrive/Option Trading/model-1m(price).h5')

## Predict

In [ ]:
np.random.seed(42)
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)
mse = np.mean(predictions - y_test)**2
mse

2.468056672920961e-08

In [ ]:
x = scaler.inverse_transform([[0.4404444]])
x

array([[1.20254334]])

## Save the model

In [ ]:
model.save("model-1m(price).h5") 1.1797762  0.36923077